🔴 Avancé | ⏱ 45 min | 🔑 Concepts : pre-commit, hooks, CI/CD integration

# 05 - Pre-commit Hooks et CI/CD

## Objectifs

- Comprendre les **Git hooks** et leur utilité
- Maîtriser le framework **pre-commit**
- Configurer des hooks essentiels pour la qualité du code
- Intégrer pre-commit dans un workflow CI/CD
- Créer des hooks personnalisés

## Prérequis

- Git installé et configuré
- Connaissance de base de Git (commit, push, etc.)
- Familiarité avec ruff et mypy (voir notebook 01)

## 1. Qu'est-ce qu'un Git hook ?

Les **Git hooks** sont des scripts qui s'exécutent automatiquement à certains moments du workflow Git.

### Hooks les plus courants

| Hook | Quand ? | Utilisation |
|------|---------|-------------|
| `pre-commit` | Avant un commit | Vérifier le style, formater le code |
| `commit-msg` | Avant d'enregistrer le message | Valider le format du message |
| `pre-push` | Avant un push | Exécuter les tests |
| `post-merge` | Après un merge | Installer les dépendances |

### Où sont les hooks ?

```bash
.git/hooks/
├── pre-commit.sample
├── pre-push.sample
└── ...
```

**Problème** : Les hooks dans `.git/hooks/` ne sont pas versionnés !

**Solution** : Le framework **pre-commit** !

## 2. Pre-commit framework

**Pre-commit** est un framework qui gère les hooks Git de façon versionnée et partagée.

In [ ]:
# Installation
!pip install pre-commit -q

In [ ]:
# Vérifier l'installation
!pre-commit --version

### Installation dans un projet Git

In [ ]:
# Créer un projet de démonstration
!mkdir -p demo_precommit
!cd demo_precommit && git init
print("Projet Git initialisé")

In [ ]:
%%writefile demo_precommit/.pre-commit-config.yaml
# Configuration pre-commit basique
repos:
  - repo: https://github.com/pre-commit/pre-commit-hooks
    rev: v4.5.0
    hooks:
      - id: trailing-whitespace
      - id: end-of-file-fixer
      - id: check-yaml
      - id: check-added-large-files

In [ ]:
# Installer les hooks dans le projet
!cd demo_precommit && pre-commit install

## 3. Configuration .pre-commit-config.yaml

Le fichier `.pre-commit-config.yaml` définit tous les hooks à exécuter.

In [ ]:
%%writefile .pre-commit-config.yaml
# Configuration pre-commit complète pour Python
repos:
  # Hooks génériques
  - repo: https://github.com/pre-commit/pre-commit-hooks
    rev: v4.5.0
    hooks:
      # Supprime les espaces en fin de ligne
      - id: trailing-whitespace
      # Ajoute une ligne vide à la fin des fichiers
      - id: end-of-file-fixer
      # Vérifie la syntaxe YAML
      - id: check-yaml
      # Vérifie la syntaxe JSON
      - id: check-json
      # Vérifie la syntaxe TOML
      - id: check-toml
      # Détecte les gros fichiers (> 500KB par défaut)
      - id: check-added-large-files
        args: ['--maxkb=500']
      # Détecte les clés privées commitées par erreur
      - id: detect-private-key
      # Empêche de commiter sur main/master
      - id: no-commit-to-branch
        args: ['--branch', 'main', '--branch', 'master']
      # Vérifie les conflits de merge
      - id: check-merge-conflict
      # Vérifie la syntaxe Python
      - id: check-ast
      # Fixe l'encoding des fichiers Python
      - id: fix-encoding-pragma
        args: ['--remove']

  # Ruff : linting et formatage
  - repo: https://github.com/astral-sh/ruff-pre-commit
    rev: v0.1.9
    hooks:
      # Linting
      - id: ruff
        args: [--fix, --exit-non-zero-on-fix]
      # Formatage
      - id: ruff-format

  # Mypy : vérification de types
  - repo: https://github.com/pre-commit/mirrors-mypy
    rev: v1.8.0
    hooks:
      - id: mypy
        additional_dependencies: [types-requests]

  # Vérifications de sécurité
  - repo: https://github.com/PyCQA/bandit
    rev: 1.7.6
    hooks:
      - id: bandit
        args: [-c, pyproject.toml]
        additional_dependencies: ["bandit[toml]"]

  # Vérifier les requirements.txt
  - repo: https://github.com/pycqa/pip-audit
    rev: v2.6.3
    hooks:
      - id: pip-audit

## 4. Hooks essentiels pour Python

### 4.1 Trailing whitespace et end-of-file

In [ ]:
%%writefile example_whitespace.py
# Fichier avec problèmes de formatting
def hello():    
    print("hello")    
    # Espaces en fin de ligne ci-dessus
# Pas de ligne vide à la fin

### 4.2 Ruff (linting + formatting)

In [ ]:
%%writefile example_ruff.py
# Code avec problèmes détectables par ruff
import os
import sys

def mauvaise_fonction(x,y):
    unused_var = 42
    result=x+y
    return result

### 4.3 Mypy (type checking)

In [ ]:
%%writefile example_mypy.py
# Code avec erreurs de types
def addition(a: int, b: int) -> int:
    return a + b

# Erreur de type détectable par mypy
result = addition(5, "10")  # str au lieu de int

### 4.4 Détection de secrets

In [ ]:
%%writefile example_secret.py
# ⚠️ DANGER : Code avec clé privée
API_KEY = "sk_live_1234567890abcdefghij"  # Sera détecté
PASSWORD = "super_secret_password_123"   # Sera détecté

# ✅ BON : Utiliser des variables d'environnement
import os
API_KEY = os.getenv("API_KEY")

## 5. Exécution des hooks

### Commandes pre-commit

In [ ]:
%%writefile demo_file.py
import os
import sys

def test():    
    unused = 42
    print("test")

In [ ]:
# Exécuter les hooks sur tous les fichiers
!pre-commit run --all-files

In [ ]:
# Exécuter un hook spécifique
!pre-commit run ruff --all-files

In [ ]:
# Exécuter seulement sur les fichiers stagés (comportement par défaut)
# pre-commit run

### Workflow typique

```bash
# 1. Modifier du code
vim mon_fichier.py

# 2. Stager les changements
git add mon_fichier.py

# 3. Commiter (les hooks s'exécutent automatiquement)
git commit -m "Mon message"

# Si un hook échoue :
# - Le commit est annulé
# - Les fichiers sont modifiés automatiquement (si possible)
# - Vous devez re-stager et re-commiter

# 4. Re-stager après correction automatique
git add mon_fichier.py

# 5. Re-commiter
git commit -m "Mon message"
```

## 6. pre-commit autoupdate

Mettre à jour automatiquement les versions des hooks.

In [ ]:
# Mettre à jour toutes les révisions
!pre-commit autoupdate

Cela met à jour le fichier `.pre-commit-config.yaml` avec les dernières versions disponibles.

## 7. Hooks personnalisés

Vous pouvez créer vos propres hooks locaux.

In [ ]:
%%writefile .pre-commit-config-custom.yaml
repos:
  # Hooks locaux personnalisés
  - repo: local
    hooks:
      # Vérifier qu'il n'y a pas de print() dans le code
      - id: no-print-statements
        name: Détecte les print()
        entry: python -c "import sys; sys.exit(any('print(' in line for line in open(f).readlines()) for f in sys.argv[1:])"
        language: system
        files: \.py$
      
      # Vérifier que les tests sont présents
      - id: ensure-tests-exist
        name: Vérifie qu'un fichier test existe
        entry: ./scripts/check_tests.sh
        language: script
        files: \.py$
        pass_filenames: false
      
      # Exécuter pytest avant commit
      - id: pytest-check
        name: Exécute pytest
        entry: pytest
        language: system
        pass_filenames: false
        always_run: true

In [ ]:
%%writefile scripts/check_tests.sh
#!/bin/bash
# Script pour vérifier la présence de tests

# Compter les fichiers Python (hors tests)
py_files=$(find . -name "*.py" -not -path "*/test_*" -not -path "*/tests/*" | wc -l)

# Compter les fichiers de tests
test_files=$(find . -name "test_*.py" -o -path "*/tests/*.py" | wc -l)

if [ $test_files -eq 0 ] && [ $py_files -gt 0 ]; then
    echo "❌ Aucun fichier de test trouvé !"
    exit 1
fi

echo "✅ Fichiers de tests présents ($test_files fichiers)"
exit 0

## 8. Intégration CI/CD : GitHub Actions

Exécuter pre-commit dans votre pipeline CI/CD.

In [ ]:
%%writefile .github/workflows/pre-commit.yml
name: Pre-commit Checks

on:
  push:
    branches: [main, develop]
  pull_request:
    branches: [main, develop]

jobs:
  pre-commit:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      
      - name: Set up Python
        uses: actions/setup-python@v5
        with:
          python-version: '3.11'
      
      - name: Install pre-commit
        run: pip install pre-commit
      
      - name: Run pre-commit
        run: pre-commit run --all-files

### Workflow CI/CD complet

In [ ]:
%%writefile .github/workflows/ci.yml
name: CI

on:
  push:
    branches: [main]
  pull_request:
    branches: [main]

jobs:
  pre-commit:
    name: Pre-commit hooks
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      - uses: actions/setup-python@v5
        with:
          python-version: '3.11'
      - uses: pre-commit/action@v3.0.0

  tests:
    name: Tests (Python ${{ matrix.python-version }})
    runs-on: ubuntu-latest
    strategy:
      matrix:
        python-version: ['3.9', '3.10', '3.11']
    
    steps:
      - uses: actions/checkout@v4
      
      - name: Set up Python ${{ matrix.python-version }}
        uses: actions/setup-python@v5
        with:
          python-version: ${{ matrix.python-version }}
      
      - name: Install dependencies
        run: |
          python -m pip install --upgrade pip
          pip install -e ".[dev]"
      
      - name: Run tests with coverage
        run: |
          pytest --cov=src --cov-report=xml --cov-report=term
      
      - name: Upload coverage to Codecov
        uses: codecov/codecov-action@v3
        with:
          file: ./coverage.xml
          fail_ci_if_error: true

  security:
    name: Security checks
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      - uses: actions/setup-python@v5
        with:
          python-version: '3.11'
      
      - name: Run bandit
        run: |
          pip install bandit[toml]
          bandit -r src/ -c pyproject.toml
      
      - name: Run safety
        run: |
          pip install safety
          safety check

## 9. Configuration avancée

### Exclure des fichiers

In [ ]:
%%writefile .pre-commit-config-exclude.yaml
repos:
  - repo: https://github.com/astral-sh/ruff-pre-commit
    rev: v0.1.9
    hooks:
      - id: ruff
        # Exclure certains fichiers
        exclude: '^(migrations/|tests/fixtures/)'
      
      - id: ruff-format
        # Exclure par pattern
        exclude: '.*_pb2\.py$'  # Fichiers protobuf générés

### Arguments personnalisés

In [ ]:
%%writefile .pre-commit-config-args.yaml
repos:
  - repo: https://github.com/astral-sh/ruff-pre-commit
    rev: v0.1.9
    hooks:
      - id: ruff
        args:
          - --fix
          - --exit-non-zero-on-fix
          - --select=E,W,F,I
  
  - repo: https://github.com/pre-commit/mirrors-mypy
    rev: v1.8.0
    hooks:
      - id: mypy
        args:
          - --strict
          - --ignore-missing-imports

## Pièges courants

### 1. Abus de --no-verify

```bash
# ❌ MAUVAIS : Bypasser systématiquement les hooks
git commit --no-verify -m "Quick fix"

# ✅ BON : Utiliser --no-verify seulement en cas d'urgence
# Et créer un ticket pour corriger le problème
```

### 2. Hooks trop lents

```yaml
# ❌ MAUVAIS : Exécuter tous les tests à chaque commit
- repo: local
  hooks:
    - id: pytest-all
      name: Run all tests
      entry: pytest tests/  # Peut prendre 10 minutes !
      language: system

# ✅ BON : Tests rapides en pre-commit, tests complets en CI
- repo: local
  hooks:
    - id: pytest-fast
      name: Run fast tests only
      entry: pytest tests/ -m "not slow"
      language: system
```

### 3. Conflits avec la CI

```bash
# ❌ PROBLÈME : pre-commit passe localement mais échoue en CI
# Cause : versions différentes des outils

# ✅ SOLUTION : Utiliser les mêmes versions
# 1. Fixer les versions dans .pre-commit-config.yaml
# 2. Utiliser pre-commit autoupdate régulièrement
# 3. Exécuter pre-commit en CI avec la même config
```

### 4. Oublier de partager les hooks

```bash
# ❌ PROBLÈME : Nouveaux développeurs n'ont pas les hooks

# ✅ SOLUTION : Documentation dans README.md
# "## Installation
# pip install pre-commit
# pre-commit install
# "

# Ou script d'installation automatique
```

## Mini-Exercices

### Exercice 1 : Configurer pre-commit pour un projet

Créez une configuration `.pre-commit-config.yaml` qui :
- Utilise ruff pour le linting et le formatage
- Utilise mypy pour le type checking
- Vérifie les trailing whitespace
- Détecte les gros fichiers (> 1MB)
- Empêche de commiter sur main

In [ ]:
# Votre configuration ici


### Exercice 2 : Créer un hook personnalisé

Créez un hook local qui vérifie que tous les fichiers Python ont un docstring.

In [ ]:
# Votre hook ici


### Exercice 3 : GitHub Actions workflow

Créez un workflow GitHub Actions qui exécute pre-commit sur toutes les PR.

In [ ]:
# Votre workflow ici


## Solutions

### Solution Exercice 1

In [ ]:
%%writefile .pre-commit-config-ex1.yaml
repos:
  - repo: https://github.com/pre-commit/pre-commit-hooks
    rev: v4.5.0
    hooks:
      - id: trailing-whitespace
      - id: end-of-file-fixer
      - id: check-added-large-files
        args: ['--maxkb=1024']  # 1MB
      - id: no-commit-to-branch
        args: ['--branch', 'main']

  - repo: https://github.com/astral-sh/ruff-pre-commit
    rev: v0.1.9
    hooks:
      - id: ruff
        args: [--fix, --exit-non-zero-on-fix]
      - id: ruff-format

  - repo: https://github.com/pre-commit/mirrors-mypy
    rev: v1.8.0
    hooks:
      - id: mypy
        args: [--strict]

### Solution Exercice 2

In [ ]:
%%writefile check_docstrings.py
#!/usr/bin/env python3
"""Vérifie que tous les fichiers Python ont un docstring."""
import ast
import sys

def check_docstring(filename):
    """Vérifie qu'un fichier a un docstring."""
    with open(filename, 'r') as f:
        try:
            tree = ast.parse(f.read())
        except SyntaxError:
            return True  # Ignore syntax errors (handled by other tools)
    
    # Vérifier le docstring du module
    docstring = ast.get_docstring(tree)
    if not docstring:
        print(f"❌ {filename}: manque un docstring de module")
        return False
    
    return True

def main():
    """Point d'entrée."""
    files = sys.argv[1:]
    all_good = all(check_docstring(f) for f in files)
    sys.exit(0 if all_good else 1)

if __name__ == "__main__":
    main()

In [ ]:
%%writefile .pre-commit-config-ex2.yaml
repos:
  - repo: local
    hooks:
      - id: check-docstrings
        name: Vérifie les docstrings
        entry: python check_docstrings.py
        language: system
        files: \.py$
        exclude: '^tests/'

### Solution Exercice 3

In [ ]:
%%writefile .github/workflows/pre-commit-ex3.yml
name: Pre-commit

on:
  pull_request:
    branches: [main, develop]

jobs:
  pre-commit:
    name: Run pre-commit hooks
    runs-on: ubuntu-latest
    
    steps:
      - name: Checkout code
        uses: actions/checkout@v4
      
      - name: Set up Python
        uses: actions/setup-python@v5
        with:
          python-version: '3.11'
      
      - name: Cache pre-commit
        uses: actions/cache@v3
        with:
          path: ~/.cache/pre-commit
          key: pre-commit-${{ hashFiles('.pre-commit-config.yaml') }}
      
      - name: Run pre-commit
        uses: pre-commit/action@v3.0.0
      
      - name: Annotate with errors
        if: failure()
        run: |
          echo "::error::Pre-commit hooks failed. Please run 'pre-commit run --all-files' locally."